In [1]:
import networkx as nx
import simplejson as json
from networkx.readwrite import json_graph

In [4]:
g

NameError: name 'g' is not defined

In [6]:
# parse the gml file and build the graph object
g = nx.read_gml('networks/data/power.gml', label='id')
# create a dictionary in a node-link format that is suitable for JSON serialization
d = json_graph.node_link_data(g)
with open('networks/data/power_grid_westernus.json', 'w') as fp:
    json.dump(d, fp)

In [ ]:
g.plt.show()